# Machine Learning - Análise de Sentimento.

Para essa atividade vamos gerar uma modelo de análise de sentimento em inglês baseado em reviews retirados de 3 sites: Amazon, IMDb e Yelp. Essa base está disponível [neste link](https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences). Mais detalhes podem ser encontrados no link ou no artigo de referência: *From Group to Individual Labels using Deep Features', Kotzias et. al,. KDD 2015*. 

A base possui um texto e para cada texto um sentimento sobre o conteúdo abordado no texto. Os sentimentos podem ser positivos (1) ou negativos (2). Foram coletados em média 500 textos para cada sentimento em cada base. 

Nesse notebook vamos construir uma modelo de aprendizagem para análise de sentimento em inglês. O primeiro passo foi carregar o Dataset de forma apropriada e em seguida construir a matriz de entrada para nosso algoritmo de Machine Learning.

## Carregando o Dataset

In [2]:
import pandas as pd
from joblib import dump, load
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [3]:


df_amazon = pd.read_csv("../datasets/analise_de_sentimento/amazon_cells_labelled.txt", 
                        sep="\t", header=None, names=['Text','Sentiment'])
df_imdb = pd.read_csv("../datasets/analise_de_sentimento/imdb_labelled.txt", 
                        sep="\t", header=None, names=['Text','Sentiment'])
df_yelp = pd.read_csv("../datasets/analise_de_sentimento/yelp_labelled.txt", 
                        sep="\t", header=None, names=['Text','Sentiment'])

join_frames = [df_amazon, df_imdb, df_yelp]

df_final_dataset = pd.concat(join_frames)



In [4]:
df_final_dataset

,Text,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1
...,...,...
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0


## Construindo a base de dados

A base de dados possui 2748 textos que foram classificados em dois sentimentos: negativo (0) e positivo (1). Construa uma base de dados apropriada para os testes. Divida a base em treino e teste (80% para treino e 20% para teste). A base de treinamento será utilizado para a construção do modelo e a de teste para o teste final do modelo construído. 

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df_final_dataset['Text'], 
                                                    df_final_dataset['Sentiment'], 
                                                    random_state=42,
                                                    test_size=0.2
                                                   )
count_vector = CountVectorizer()
training_data = count_vector.fit_transform(X_train)
testing_data = count_vector.transform(X_test)

## Treinando e Salvando um modelo de Machine Learning

Nessa etapa vamos utilizar a biblioteca `scikit-learn` para treinar um modelo de Machine Learning. Em seguida vamos testar para ver a qualidade deste modelo. 

### Treinamento do Modelo 

In [7]:
naive_bayes = MultinomialNB()
naive_bayes.fit(training_data, y_train)

MultinomialNB()

### Teste do Modelo

Nesse caso, estamos olhando somente para acurácia.

In [8]:
acc_ = naive_bayes.score(testing_data, y_test)
acc_str = "Acurácia de %.2f" % (acc_ * 100)
print(acc_str)

Acurácia de 79.45


### Testando o modelo treinado com uma frase

In [9]:
text = "I dont't like that music"
predict_text = count_vector.transform([text])
naive_bayes.predict(predict_text)

array([0])

### Treinando e salvando o modelo completo 

In [10]:
count_vector = CountVectorizer()
final_data = count_vector.fit_transform(df_final_dataset['Text'])
naive_bayes = MultinomialNB()
naive_bayes.fit(final_data, df_final_dataset['Sentiment'] )

MultinomialNB()

#### Salvando o Modelo

In [11]:
model_ = {
    'model': naive_bayes,
    'vector': count_vector
}
dump(model_, "../output/model.joblib")

['../output/model.joblib']

#### Testando o modelo salvo

In [12]:
loaded_model = load("../output/model.joblib")

In [15]:
newtext = "I love you my darling."
ptext = loaded_model['vector'].transform([newtext])
loaded_model['model'].predict(ptext)

array([1])

O arquivo `model.joblib` vai ser carregado no bot para usar a funcionalidade de análise de sentimento.